In [1]:
# Environment Variable (with dotenv)
import dotenv
import os
dotenv.load_dotenv()
import openai
import dotenv
import os
dotenv.load_dotenv()
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

True

In [3]:
%pip install --upgrade --quiet  langchain langchain-openai

DEPRECATION: Loading egg at /Users/anushka/.local/lib/python3.11/site-packages/mylinear_cpp-0.0.0-py3.11-macosx-13.0-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [82]:
from aiconfig import AIConfigRuntime
import aiconfig_extension_gemini
# Load the config you just created
from aiconfig import CallbackManager, InferenceOptions
aiconfig = AIConfigRuntime.load("interview.aiconfig.json")

aiconfig.callback_manager = CallbackManager([]) # Skip Logging, Google Colab forwards logs to stdout
inference_options = InferenceOptions(stream=True)

In [83]:
memory = ConversationBufferWindowMemory(human_prefix = "Question:",ai_prefix = "Candidate Response:",k=2)

In [84]:
a=aiconfig.get_parameters("response_feedback")
candidate_response=a['candidate_response']
candidate_emotion_analysis=a['candidate_emotion_analysis']
aiconfig.delete_output("response_feedback")
#await aiconfig.run("response_feedback", params={"candidate_response": candidate_response,"candidate_emotion_analysis":candidate_emotion_analysis})
await aiconfig.run("response_feedback")
output_text = aiconfig.get_output_text("response_feedback")
print(f'Gemini output: {output_text}')

Gemini output: **Emotional Insight**

* The candidate seems confident and well-prepared for the interview.
* They maintain a positive and enthusiastic demeanor throughout their response.
* Their body language is relaxed and open, and they make good eye contact with the interviewer.

**Content Accuracy**

* The candidate's response is generally accurate and relevant to the question.
* They provide a concise overview of their education, work experience, and research interests.
* However, they could have provided more specific examples of their work experience and how it relates to the role they are interviewing for.

**Clarity and Structure**

* The candidate's response is well-organized and easy to follow.
* They use clear and concise language, and they provide a logical flow of ideas.

**Sufficiency and Depth**

* The candidate's response provides a good overview of their qualifications and experience.
* However, they could have provided more in-depth information about their skills and

In [85]:
inputs={"inputs":a['question']}
outputs={"outputs":a['candidate_response']}
memory.save_context(inputs, outputs)

In [86]:
interview_history=memory.load_memory_variables({})['history']
interview_history

'Question:: tell me about yourself.\nCandidate Response:: Hi, I am Anushka. I am MSCS student at Umass Amherst.I have 2 years of experience as ML Engineer in India. I have worked at a Automated Product cataloging startup for 2 years and then at PwC as an ML associate for an year. Most of my workex has focused on Text generation, finetunig transformer models, Retrieval Augmented Generation based Pipeline over Audit data and a little weak supervision work. My interests currently include NLP and Information retrieval systems. I am also working at the BIONLP lab at Umass where I am working on LLM Alignment and Self rewarding LLMs in medical domain.'

In [87]:
aiconfig.delete_output("QG_prompt")

[ExecuteResult(output_type='execute_result', execution_count=0, data='**Next Question:**\n\n"Anushka, in your previous role at PwC, you mentioned working on a prototype of the Audit Assistant Platform using various NLP techniques. Can you describe the specific challenges you faced during the development process and how you overcame them? This will help us understand your problem-solving abilities and how you approach technical complexities in your work."', mime_type=None, metadata={'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]})]

In [90]:
aiconfig.get_parameters("QG_prompt")

{'candidate_profile': 'Anushka Yadav is a highly skilled computer science graduate student at the University of Massachusetts Amherst, with a strong background in machine learning, big data, and information retrieval. She holds a B.Tech in Computer Science and Engineering from the Indian Institute of Information Technology, Nagpur, with a CGPA of 8.21/10.\n\nWork Experience:\n- Machine Learning Associate 2 at PwC AC Bangalore (US Advisory), where she developed a prototype of the Audit Assistant Platform using multi-source document QnA system, semantic vector research, and LLMs. She also integrated a Langchain-based Chatbot and researched weak supervision in NLP using GPT-3.\n- Machine Learning Engineer at Text Mercato Solutions Private Limited, where she handled the product development lifecycle of an Automatic Product description generator, optimized the Quality Check process, and enhanced the accuracy of an AI Data Sourcing Tool.\n- Deep Learning Intern at Text Mercato Solutions Priv

In [ ]:
aiconfig.get_prompt("QG_prompt")

In [94]:

# This time, let's also add a callback manager to log more detailed events
aiconfig.callback_manager = CallbackManager([])

await aiconfig.run("QG_prompt", params={'interview_history': interview_history},run_with_dependencies=True,options=inference_options)
output_text = aiconfig.get_output_text("QG_prompt")
print(f'Gemini output: {output_text}')

Gemini output: **Next Question:**

"Given your research experience in LLM alignment and self-rewarding LLMs in the medical domain, could you elaborate on your understanding of the challenges involved in deploying and evaluating these models in a clinical setting? Specifically, how do you ensure responsible and ethical use of such models in healthcare?"


In [95]:
a

{'candidate_emotion_analysis': 'The candidate looks confident and enthusiastic with a smiling face.',
 'candidate_response': 'Hi, I am Anushka. I am MSCS student at Umass Amherst.I have 2 years of experience as ML Engineer in India. I have worked at a Automated Product cataloging startup for 2 years and then at PwC as an ML associate for an year. Most of my workex has focused on Text generation, finetunig transformer models, Retrieval Augmented Generation based Pipeline over Audit data and a little weak supervision work. My interests currently include NLP and Information retrieval systems. I am also working at the BIONLP lab at Umass where I am working on LLM Alignment and Self rewarding LLMs in medical domain.',
 'question': 'tell me about yourself.'}

In [ ]:
async def analyse_and_generate(aiconfig,candidate_emotion_analysis,candidate_response,question,time_left):
    aiconfig.delete_output("response_feedback")
    a=aiconfig.get_parameters("response_feedback")
    await aiconfig.run("response_feedback", params={"candidate_response": candidate_response,"candidate_emotion_analysis":candidate_emotion_analysis})
    response_feedback = aiconfig.get_output_text("response_feedback")
    #adding current q and r to memory
    inputs={"inputs":a['question']}
    outputs={"outputs":a['candidate_response']}
    memory.save_context(inputs, outputs)
    interview_history=memory.load_memory_variables({})['history']
    
    aiconfig.delete_output("QG_prompt")
    await aiconfig.run("QG_prompt", params={'interview_history': interview_history},run_with_dependencies=True,options=inference_options)
    question_res = aiconfig.get_output_text("QG_prompt")
    
    return question_res